# HexMagic

> Fill in a module description here

```python
#| default_exp core
```

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

In [ ]:
#| export
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.plot.primitives import  MapCord , PrimitiveDemo
from HexMagic.plot.hex import Hex


from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import HexGrid, HexPosition ,  HexRegion, GosperCurve, windy_edge , unique_windy_edge

import numpy as np

from HexMagic.terrain import Terrain
from HexMagic.voronoi import generate_plate_terrain


Terrain.fromSeeds = generate_plate_terrain
from HexMagic.climate import ClimatePreset, Climate


In [ ]:
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)

    sampleMap.colorMap()
    sgrid = sampleMap.hexGrid
    sgrid.builder.adjust("regions", sgrid.styleLayer(f=windy_edge(iterations=2, offset_factor=0.1)))
    #sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()

In [ ]:
demoTerr()

In [ ]:
from HexMagic.terrainpatterns import TerrainPatterns
from HexMagic.climate import TerrainFactory

## Climate

In [ ]:


def island_demo_fixed(debug=False):
    """Create a tropical island with three volcanoes and downsampled rivers."""
    
    # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(800, 800))
    terrain , plates = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        ocean_fraction=0.6,
        debug = debug
    )
    if debug:
        print("\n=== COMPUTING CLIMATE ===")
    terrain.climate.configure(terrain,debug=debug)
    
    
    # 6. Visualize original
    if debug:
        print("\n=== RENDERING ORIGINAL ===")
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.add_climate_overlay()
    
    
    # 7. Downsample terrain (including flow)
    if debug:
        
        print("\n=== DOWNSAMPLING ===")
    smaller = terrain.downsample_climate(0.5)
    smaller.hexGrid.adjustRadius(20)
    
    # 9. Visualize downsampled version
    smaller.colorMap()
    smaller.hexGrid.update()
    smaller.add_climate_overlay()

    return smaller

isf = island_demo_fixed()
isf.colorMap
isf.hexGrid.update()
isf.hexGrid.builder.show()



In [ ]:
isf.builder.layers = []
mountains = isf.find_peaks(7,4)
for i , epicenter in enumerate(mountains):
    isf.elevations += isf.volcano(center=epicenter, adjusted=20+ ((i+1)*30), num_rings=5)
isf.colorMap()
isf.hexGrid.update()
isf.builder.show()

## Hydrology

In [ ]:
from HexMagic.hydrology import DrainageBasins

In [ ]:
def hydrate(terrain):

    basin = DrainageBasins(terrain)

     
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )

    river_style = StyleCSS(
        "nile",
        fill = "none",
        stroke= '#23194629',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""

    mainBasins = basin.get_major(6)
    
    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        river_svg += small_river.draw()

    terrain.hexGrid.builder.adjust("rivers", river_svg)



In [ ]:
hydrate(isf)
isf.hexGrid.builder.show()

In [ ]:
??HexGrid.regions_by_value


```python
@patch
def regions_by_value(grid: HexGrid, data: np.ndarray) -> list[HexRegion]:
    """Convert data array into list of HexRegions, one per unique value.

    Returns regions in order of sorted unique values (0, 1, 2, ...).
    """
    regions = []
    for val in sorted(np.unique(data)):
        if val >= 0:
            indices = set(np.where(data == val)[0].tolist())
            regions.append(HexRegion(indices, grid))
    return regions
```

**File:** `~/HexMagic/HexMagic/plot/region.py`

In [ ]:
def showclimates(terrain):

    basin = DrainageBasins(terrain)

     
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )

    river_style = StyleCSS(
        "nile",
        fill = "none",
        stroke= '#23194629',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""

    mainBasins = basin.get_major(6)
    
    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        river_svg += small_river.draw()

    sgrid = terrain.hexGrid
    sgrid.builder.layers = []




    climate_colors = {
        Climate.MARINE.value: "#1e88e5",
        Climate.FRESHWATER.value: "#42a5f5",
        Climate.TUNDRA.value: "#e3f2fd",
        Climate.DESERT.value: "#fdd835",
        Climate.GRASSLAND.value: "#9ccc65",
        Climate.FOREST.value: "#2e7d32",
        Climate.JUNGLE.value: "#1b5e20",
    }
    
    if 'climate' not in terrain.fields:
        terrain.compute_climate()
    
    climate_indices = terrain.fields['climate'].astype(int)
    styles = [StyleCSS(f"climate_{i}",fill = climate_colors[i]) for i in range(len(Climate))]
    for style in styles:
        terrain.hexGrid.builder.add_style(style)

    
    #work
    sgrid = terrain.hexGrid
    print(climate_indices[:20])
    climateRegions = sgrid.regions_by_value(climate_indices)
    retLayer = ""
    
    borders = {}  # Shared cache across all regions
    
    overlay = ""
    for region in climateRegions:
    
        idx  =  region.hexes.pop()
        region.hexes.add(idx)
        
        styleI = int(climate_indices[idx])
        style = styles[styleI]
        print(idx,styleI,len(region.hexes))

        for path in region.trace_perimeter_cached(borders,
          style=style,
          f=unique_windy_edge(iterations=2)):
                overlay += path.svg()

    sgrid.builder.adjust("climate",overlay)
    sgrid.builder.adjust("rivers", river_svg)


In [ ]:


showclimates(isf)
isf.hexGrid.builder.show()



[0 0 0 0 0 0 0 0 0 0 0 3 5 5 5 4 3 6 0 0]
0 0 356
640 3 72
519 4 78
515 5 121
512 6 49
